In [ ]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta


In [ ]:
ticker_symbol='HDFCBANK.NS'

In [ ]:
Data=yf.download(tickers=ticker_symbol,period="2y",interval="1d",group_by=ticker_symbol)

In [10]:
Data.columns = Data.columns.droplevel(0)
Data.head(5)

Price              Open         High          Low        Close    Volume
Date                                                                    
2023-07-10  1620.793425  1636.162176  1609.766908  1616.353516  19199221
2023-07-11  1622.744978  1635.430296  1605.668587  1608.498413  25335213
2023-07-12  1615.280167  1625.965180  1587.762831  1593.422363  35479179
2023-07-13  1616.597481  1617.329326  1598.399023  1601.375122  20956773
2023-07-14  1613.572448  1613.572448  1595.422709  1604.692749  12363607

In [12]:
Data['RSI']=ta.rsi(Data["Close"], length=14)

In [18]:
info = yf.Ticker(ticker_symbol).info
Data['Sector']=info.get("sector")
Data['Industry']=info.get("industry")

In [22]:
MACD=ta.macd(Data["Close"], fast=12, slow=26, signal=9)
Data=pd.concat([Data,MACD],axis=1)

In [24]:
Data.rename(columns={
    "MACD_12_26_9": "MACD_Line",
    "MACDs_12_26_9": "MACD_Signal",
    "MACDh_12_26_9": "MACD_Histogram"
}, inplace=True)


In [30]:
Data["EMA_20"] = ta.ema(Data["Close"], length=20)
Data["EMA_50"] = ta.ema(Data["Close"], length=50)

In [33]:
# Add Bollinger Bands (20-day default)
bbands = ta.bbands(Data["Close"], length=20)

# Combine with main DataFrame
Data = pd.concat([Data, bbands], axis=1)

# Optional rename (if needed)
Data.rename(columns={
    "BBL_20_2.0": "BB_Lower",
    "BBM_20_2.0": "BB_Middle",
    "BBU_20_2.0": "BB_Upper",
    "BBB_20_2.0": "BB_Bandwidth",
    "BBP_20_2.0": "BB_Percent"
}, inplace=True)

In [35]:
# Add ADX (default 14-period)
adx_df = ta.adx(Data["High"], Data["Low"], Data["Close"], length=14)

# Join it with main DataFrame
Data = pd.concat([Data, adx_df], axis=1)

# Optional rename (for clarity)
Data.rename(columns={
    "ADX_14": "ADX",
    "DMP_14": "DI_Plus",
    "DMN_14": "DI_Minus"
}, inplace=True)

In [37]:
# Add ATR (default length=14)
Data["ATR_14"] = ta.atr(Data["High"], Data["Low"], Data["Close"], length=14)

In [38]:
# Add Stochastic Oscillator (default k=14, d=3)
stoch_df = ta.stoch(Data["High"], Data["Low"], Data["Close"], k=14, d=3)

# Combine with main DataFrame
Data = pd.concat([Data, stoch_df], axis=1)

# Optional rename
Data.rename(columns={
    "STOCHk_14_3_3": "Stoch_%K",
    "STOCHd_14_3_3": "Stoch_%D"
}, inplace=True)



In [40]:
# Add CCI (default period = 20)
Data["CCI_20"] = ta.cci(Data["High"], Data["Low"], Data["Close"], length=20)

In [41]:
Data.head(50)

Open         High          Low        Close    Volume  \
Date                                                                       
2023-07-10  1620.793425  1636.162176  1609.766908  1616.353516  19199221   
2023-07-11  1622.744978  1635.430296  1605.668587  1608.498413  25335213   
2023-07-12  1615.280167  1625.965180  1587.762831  1593.422363  35479179   
2023-07-13  1616.597481  1617.329326  1598.399023  1601.375122  20956773   
2023-07-14  1613.572448  1613.572448  1595.422709  1604.692749  12363607   
2023-07-17  1610.059669  1641.285069  1593.471175  1638.260132  24626464   
2023-07-18  1656.897692  1662.752454  1629.575468  1636.893921  40538409   
2023-07-19  1640.406884  1647.871730  1634.454565  1644.310059  16467567   
2023-07-20  1640.455695  1651.043034  1639.333509  1647.871704  28105474   
2023-07-21  1639.333402  1648.896228  1632.746795  1635.186279  21077124   
2023-07-24  1637.869772  1643.870928  1629.673081  1637.772217  16089722   
2023-07-25  1643.870929  1657.873545  1637.772218  1655.531616  27996298   
2023-07-26  1658.459002  1658.459002  1647.139819  1649.774414  12397179   
2023-07-27  1658.751774  1661.776711  1627.087196  1632.649292  29870651   
2023-07-28  1621.281329  1628.502226  1601.375112  1603.717041  20507842   
2023-07-31  1610.108523  1616.695131  1599.033169  1611.230591  17282503   
2023-08-01  1614.401945  1627.087264  1610.059710  1622.013184  17697094   
2023-08-02  1602.253364  1611.523404  1593.617612  1600.789673  14058161   
2023-08-03  1600.301798  1611.377034  1583.713304  1589.226562  28836973   
2023-08-04  1595.569209  1616.402382  1589.812001  1612.206421  18694152   
2023-08-07  1622.842551  1622.842551  1607.669030  1611.279419  14150459   
2023-08-08  1611.718516  1615.524135  1602.302201  1609.962158  21886914   
2023-08-09  1612.986970  1614.450660  1591.617064  1610.547485  16680618   
2023-08-10  1607.181159  1614.206826  1591.519621  1596.252197  25390559   
2023-08-11  1598.740524  1598.740524  1577.077831  1579.614990  14112911   
2023-08-14  1572.003683  1575.906858  1561.513901  1571.906128  15652205   
2023-08-16  1544.779013  1570.686384  1543.022656  1567.319824  17068771   
2023-08-17  1558.928043  1570.930353  1555.415209  1558.635376  26442020   
2023-08-18  1549.121404  1557.659599  1547.120979  1552.243896  12816683   
2023-08-21  1561.757901  1561.757901  1548.584686  1551.024170  10918635   
2023-08-22  1557.708280  1559.318363  1541.754076  1544.388672  16136785   
2023-08-23  1541.754121  1552.048793  1535.167513  1548.194336  18249294   
2023-08-24  1554.732157  1557.561863  1537.997271  1541.071045  21572896   
2023-08-25  1536.436060  1539.314604  1520.676968  1523.701904  15034878   
2023-08-28  1523.701945  1544.340007  1523.701945  1539.558594  14433619   
2023-08-29  1551.219238  1554.097901  1543.168940  1551.804810  17769466   
2023-08-30  1565.124265  1565.124265  1535.606505  1540.485474  14042102   
2023-08-31  1540.778301  1544.827869  1525.263157  1533.411011  50037886   
2023-09-01  1532.972008  1540.778358  1521.603987  1536.582397  21613388   
2023-09-04  1551.511998  1552.926851  1536.777537  1546.193970  12536819   
2023-09-05  1549.950803  1549.950803  1532.581627  1536.777588  19356041   
2023-09-06  1538.631611  1561.269978  1536.777579  1556.878906  28160455   
2023-09-07  1556.879011  1574.931196  1548.731156  1571.857422  17104171   
2023-09-08  1569.369176  1593.373679  1568.686048  1584.103638  17251105   
2023-09-11  1589.470467  1596.886476  1584.689054  1592.300293  15786511   
2023-09-12  1600.301792  1603.131618  1585.713723  1597.276855  19516670   
2023-09-13  1591.812444  1611.620986  1589.177729  1604.107422  23210047   
2023-09-14  1604.448890  1610.059704  1595.520353  1603.131592  11769424   
2023-09-15  1610.059571  1629.575444  1610.059571  1621.525146  86705601   
2023-09-18  1613.718879  1614.938621  1586.738158  1589.616821  18528144   

                  RSI              Sector          Industry  MACD_

In [42]:
Data.to_csv("test.csv")